In [35]:
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt
from os.path import join
from tqdm.notebook import tqdm
import os
from os.path import basename
from config import bkg_data_path, signal_data_path, processed_data_path
import time
import mplhep as hep
%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 30.8 ms (started: 2021-06-17 17:46:12 +01:00)


# Sanity checks
### Listing all the files on the bkg and signal directory (file path)

In [2]:
bkg_files = glob.glob(join(bkg_data_path, "*.*"))
signal_files = glob.glob(join(signal_data_path, "*/*.*"))
all_files = bkg_files + signal_files

print("Signal files:", len(signal_files), "\nBackground files:", len(bkg_files), "\n> Total:", len(all_files))

Signal files: 5 
Background files: 18 
> Total: 23
time: 2.49 ms (started: 2021-06-17 16:43:35 +01:00)


## Checking features

In [3]:
book = {}
for path in all_files:
    # Load data
    if path.endswith(".csv"):
        data = pd.read_csv(path)
    elif path.endswith(".h5"):
        data = pd.read_hdf(path)

    # Get features
    features = list(data.columns)

    file_name = basename(path)
    for feature in features:
        if feature not in book:
            book[feature] = []
        book[feature] += [file_name]

    # Saving memory
    del data

After getting a directory with the structure:
- {feature:\[name_of_file\]}

We can compare each of the files features to see if they match

In [31]:
for x in book:
    if len(book[x]) != len(all_files):
        print(f"\nFeature \"{x}\" is missing on",  len(all_files)-len(book[x]), "file(s).")
        print("-> Files that are missing the feature:\n\t", set([basename(x) for x in all_files]) - set(book[x]))


Feature "gen_sample" is missing on 1 file(s).
-> Files that are missing the feature:
	 {'tZFCNC.h5'}

Feature "gen_filter" is missing on 1 file(s).
-> Files that are missing the feature:
	 {'tZFCNC.h5'}

Feature "gen_decay_filter" is missing on 1 file(s).
-> Files that are missing the feature:
	 {'tZFCNC.h5'}

Feature "MissingET_Eta" is missing on 22 file(s).
-> Files that are missing the feature:
	 {'WW_2L_PTW0to250.csv', 'ZZ_2L_PTZ500.csv', 'Zbb_2L_HT0to250.csv', 'Zbb_2L_HT250to500.csv', 'ZZ_2L_PTZ0to250.csv', 'Zjj_2L_HT500.csv', 'Zbb_2L_HT500.csv', 'mch45_HG_13TeV_HG3000_HQ1000_train.csv', 'ZZ_2L_PTZ250to500.csv', 'WW_2L_PTW500.csv', 'WZ_2L_PTZ500.csv', 'WW_2L_PTW250to500.csv', 'mch45_HG_13TeV_HG3000_HQ1000_test.csv', 'mch45_HG_13TeV_wohg_HQ1000_train.csv', 'WZ_2L_PTZ0to250.csv', 'mch45_HG_13TeV_wohg_HQ1000_test.csv', 'WZ_2L_PTZ250to500.csv', 'Zjj_2L_HT250to500.csv', 'ttbar_2L_PTtop0to100.csv', 'ttbar_2L_PTtop100to250.csv', 'Zjj_2L_HT0to250.csv', 'ttbar_2L_PTtop250.csv'}


Since the features highlighted by out little script are irrelevant, we can just mass-delete them on all the files when we pre-process the data.

# Pre-Processing
-> Delete irrelevant features (ex: features that were used on the simulation of the data)

-> Apply Cuts

- At least two final state leptons

- At least one b-tagged jet

- Large scalar sum of transverse momentum (p_t), H_t > 500 GeV

-> Weight the data


In [5]:
# This features shall be removed
to_remove = [
        'gen_xsec',
        'gen_decay1',
        'gen_decay2',
        'gen_sample',
        'gen_filter',
        'gen_decay_filter',
        'MissingET_Eta'
            ]

time: 352 µs (started: 2021-06-17 16:43:55 +01:00)


### Pre-process and join all the background data

In [6]:
def pre_process(paths):
    print("[Info] Started preprocessing..")
    keys = set([x.split("/")[-2] for x in all_files])
    print(f"[Info] Keys: {keys}\n")

    for i, path in enumerate(tqdm(paths)):
        # Load data
        if path.endswith(".csv"):
            data = pd.read_csv(path)
        elif path.endswith(".h5"):
            data = pd.read_hdf(path)

        # Add a column with the file name for later reference
        data["name"] = basename(path)

        # Weights
        data["gen_xsec"] = data["gen_xsec"].mean() / data.shape[0]
        data.rename(columns={"gen_xsec":"weights"}, inplace=True)

        # Remove ussless features
        for x in to_remove:
            try:
                data.drop([x], axis=1, inplace=True)
            except:
                pass

        # Statistical purposes
        shape_before = data.shape[0]

        ## Apply Cuts
        # At least 2 leptons
        data = data[(data['Electron_Multi'] + data['Muon_Multi']) >= 2] 
        # At least 1 B-Tag
        data = data[(data['Jet1_BTag'] + data['Jet2_BTag'] + data['Jet3_BTag'] + data['Jet4_BTag'] + data['Jet5_BTag']) >= 1]
        # H_T > 500 GeV
        data = data[data['ScalarHT_HT'] > 500]

        # Statistical purposes
        shape_after = data.shape[0]

        print(f"[Info] Data Reduction for \"{basename(path)}\": {int(((shape_before-shape_after)/shape_before)*100)}%")


        ## SAVING
        save_path = join(processed_data_path,path.split("/")[-2] + ".csv")

        # Create / Reset file on first iteration # TODO: COMPOR
        if i == 0:
            with open(save_path, 'w') as fp:
                pass

        with open(save_path, 'a') as f:
            data.to_csv(f, header=f.tell()==0)


time: 798 µs (started: 2021-06-17 16:43:55 +01:00)


In [7]:
pre_process(all_files)

[Info] Started preprocessing..
[Info] Keys: {'VLQ_HG', 'VLQ_SEM_HG', 'bkg', 'FCNC'}



  0%|          | 0/23 [00:00<?, ?it/s]

[Info] Data Reduction for "ttbar_2L_PTtop0to100.csv": 98%
[Info] Data Reduction for "ttbar_2L_PTtop100to250.csv": 96%
[Info] Data Reduction for "ttbar_2L_PTtop250.csv": 83%
[Info] Data Reduction for "WW_2L_PTW0to250.csv": 99%
[Info] Data Reduction for "WW_2L_PTW250to500.csv": 99%
[Info] Data Reduction for "WW_2L_PTW500.csv": 98%
[Info] Data Reduction for "WZ_2L_PTZ0to250.csv": 99%
[Info] Data Reduction for "WZ_2L_PTZ250to500.csv": 94%
[Info] Data Reduction for "WZ_2L_PTZ500.csv": 93%
[Info] Data Reduction for "Zbb_2L_HT0to250.csv": 99%
[Info] Data Reduction for "Zbb_2L_HT250to500.csv": 73%
[Info] Data Reduction for "Zbb_2L_HT500.csv": 61%
[Info] Data Reduction for "Zjj_2L_HT0to250.csv": 99%
[Info] Data Reduction for "Zjj_2L_HT250to500.csv": 96%
[Info] Data Reduction for "Zjj_2L_HT500.csv": 94%
[Info] Data Reduction for "ZZ_2L_PTZ0to250.csv": 99%
[Info] Data Reduction for "ZZ_2L_PTZ250to500.csv": 89%
[Info] Data Reduction for "ZZ_2L_PTZ500.csv": 90%
[Info] Data Reduction for "tZFCNC.h5"